# Getting started: Cifar-10 on Hops Notebook
---

In this notebook we are going to master running Deep Learning code on Hops using Jupyter notebooks and HopsFS, the highly efficient distributed file system that Hops provides. HopsFS is a fork of Apache HDFS and operations for reading and writing files to HopsFS is the same as with HDFS that TensorFlow supports.
For more information about how to read from HDFS, please see: https://www.tensorflow.org/deploy/hadoop

In this example program we are going to:
- Run Cifar-10 code using `tf.contrib.learn.Experiment` and `tf.estimator.Estimator`
- Using the hops python library to run TensorFlow on the Hops Platform
- Read data from a Dataset which is located in your project, on HopsFS (HDFS)
- Define a convolutional neural network
- Define a set of hyperparameters and values to perform gridsearch on
- Run training for each combination of hyperparameters using the Experiment API
- Monitor training using TensorBoard to see accuracy and loss for every hyperparameter configuration
- Monitoring training by looking at logs
- Find where the model and TensorBoard events for each hyperparameter configuration is saved in your project
- Visualize TensorBoard events from the previous TensorBoard run

## Table of contents:

### [TensorFlow on Hops](#paradigm)
### [Training Cifar-10 on Hops](#cifar10)
### [Hyperparameter search](#hyperparam)
### [Launching TensorFlow jobs](#starting)
### [Monitoring execution - TensorBoard](#tensorboard)
### [Monitoring execution - Logs](#logs)
### [Increasing throughput - Running jobs in parallel](#parallel)
### [Show TensorBoard from previous runs](#visualize)


## Hops TensorFlow programming paradigm <a class="anchor" id='paradigm'></a>

To be able to run your TensorFlow code on Hops, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function. If you wish to run gridsearch over a given set of hyperparameters, you can define arguments for this wrapper function that corresponds to the name of your hyperparameters.

You can also submit one or more `.py`, `.zip` or `.egg` files that contain your code and import them in the wrapper function. To include files, navigate back to HopsWorks and restart restart Jupyter, you can then include files in the Jupyter configuration.

## The `hops` python module

Below you can see the aforementioned wrapper function, which is coincidently named `wrapper` but could potentially be named anything. You can see two imports from the `hops` module, a `tensorboard` and an `hdfs` module. These are the only two modules that you will need to use in your TensorFlow wrapper function. 

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project in HopsFS after each hyperparameter configuration is finished. The `tflaunch.launch` function, that we will look at abit further down will return the exact path, which you can then navigate to using HopsWorks to inspect the files.

The directory could in practice be used to store other data that should be accessible after each hyperparameter configuration is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```


### Using the `hdfs` module
The `hdfs` module provides a single method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a cifar10 folder in your Resources Dataset, which is created automatically for each project, the path to the cifar10 data would be `hdfs.project_path() + 'Resources/cifar10`
```python
# Use this module to get the path to your project in HopsFS, 
# then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

![image11-Dataset-ProjectPath.png](../images/datasets.png)

## Training Cifar10 on Hops <a class="anchor" id='cifar10'></a>

### The wrapper function

Here we define the aforementioned wrapper function containing the code to run, with the hyperparameter arguments `learning_rate` and `dropout`. It simply contains all the TensorFlow code that we want to run.

In [ ]:
# Wrap the TensorFlow code you want to run in a function
# To perform hyper-parameter searching define your parameters as arguments

def wrapper(learning_rate, dropout):
    
    import tensorflow as tf
    
    # Use this module to get the TensorBoard logdir
    from hops import tensorboard
    
    # Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
    from hops import hdfs
    
    num_classes = 10 # CIFAR10 total classes (0-9 objects)
    num_steps = 200
    batch_size = 128

    # Network Parameters
    num_input = 32*32*3 # CIFAR10 data input (img shape: 32*32*3)
    
    # Create the neural network
    # TF Estimator input is a dict, in case of multiple inputs
    def conv_net(x, n_classes, dropout, reuse, is_training):
        

        # Define a scope for reusing the variables
        with tf.variable_scope('ConvNet', reuse=reuse):

            # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
            # Reshape to match picture format [Height x Width x Channel]
            # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
            x = tf.reshape(x, shape=[-1, 32, 32, 3])
            #x = tf.image.crop_and_resize(tf.reshape(x, shape=[-1, 32, 32, 3]),size=[24,24])

            # Convolution Layer with 32 filters and a kernel size of 5
            conv1 = tf.layers.conv2d(x, 64, 5, activation=tf.nn.relu)
            # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
            conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

            # Convolution Layer with 32 filters and a kernel size of 5
            conv2 = tf.layers.conv2d(conv1, 64, 5, activation=tf.nn.relu)
            # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
            conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

            # Flatten the data to a 1-D vector for the fully connected layer
            fc1 = tf.contrib.layers.flatten(conv2)

            # Fully connected layer (in tf contrib folder for now)
            fc1 = tf.layers.dense(fc1, 384)
            # Apply Dropout (if is_training is False, dropout is not applied)
            fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

            fc2 = tf.layers.dense(fc1, 192)
            # Apply Dropout (if is_training is False, dropout is not applied)
            fc2 = tf.layers.dropout(fc2, rate=dropout, training=is_training)
            
            # Output layer, class prediction
            out = tf.layers.dense(fc2, n_classes)

            return out
    
    # Define the model function (following TF Estimator Template)
    def model_fn(features, labels, mode, params):

        # Build the neural network
        # Because Dropout have different behavior at training and prediction time, we
        # need to create 2 distinct computation graphs that still share the same weights.
        logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
        logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)

        # Predictions
        pred_classes = tf.argmax(logits_test, axis=1)
        pred_probas = tf.nn.softmax(logits_test)

        # If prediction mode, early return
        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())

        # Evaluate the accuracy of the model
        acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

        image = tf.reshape(features[:10], [-1, 32, 32, 3])
        tf.summary.image("image", image)

        # TF Estimators requires to return a EstimatorSpec, that specify
        # the different ops for training, evaluating, ...
        estim_specs = tf.estimator.EstimatorSpec(
          mode=mode,
          predictions=pred_classes,
          loss=loss_op,
          train_op=train_op,
          eval_metric_ops={'accuracy': acc_op})

        return estim_specs
    
    def data_input_fn(filenames, num_input, batch_size=1000, shuffle=False, repeat=None):
    
        def parser(serialized_example):
            """Parses a single tf.Example into image and label tensors."""
            features = tf.parse_single_example(
                serialized_example,
                features={
                    'image': tf.FixedLenFeature([], tf.string),
                    'label': tf.FixedLenFeature([], tf.int64),
                })
            image = tf.decode_raw(features['image'], tf.uint8)
            image.set_shape([num_input])

            image = tf.cast(
                tf.transpose(tf.reshape(image, [3, 32, 32]), [1, 2, 0]),
                tf.float32)

            # Normalize the values of the image from the range [0, 255] to [-0.5, 0.5]
            image = tf.cast(image, tf.float32) / 255 - 0.5
            label = tf.cast(features['label'], tf.int32)
            return image, label

        def _input_fn():
            # Import CIFAR10 data
            dataset = tf.contrib.data.TFRecordDataset(filenames)

            # Map the parser over dataset, and batch results by up to batch_size
            dataset = dataset.map(parser, num_threads=1, output_buffer_size=batch_size)
            if shuffle:
                dataset = dataset.shuffle(buffer_size=256)
            dataset = dataset.batch(batch_size)
            dataset = dataset.repeat(repeat)
            iterator = dataset.make_one_shot_iterator()

            features, labels = iterator.get_next()

            return features, labels

        return _input_fn
    
    # Path to TensorBoard logdir on executor
    # Its contents will be placed in your project path /Logs/TensorFlow/appId/runId/ after the job is finished
    logdir = tensorboard.logdir()
    
    # Path to your project in HopsFS, parent folder for your DataSets (Resources, Logs etc)
    data_dir = hdfs.project_path()
    train_filenames = [data_dir + "TestJob/data/cifar10/train/train.tfrecords"]
    validation_filenames = [data_dir + "TestJob/data/cifar10/validation/validation.tfrecords"]

    run_config = tf.contrib.learn.RunConfig(
        model_dir=logdir,
        log_device_placement=True,
        save_checkpoints_steps=20,
        save_summary_steps=10,
        log_step_count_steps=10)
    
    hparams = tf.contrib.training.HParams(
        learning_rate=learning_rate, dropout_rate=dropout)

    summary_hook = tf.train.SummarySaverHook(
          save_steps = run_config.save_summary_steps,
          scaffold= tf.train.Scaffold(),
          summary_op=tf.summary.merge_all())

    mnist_estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        config=run_config,
        params=hparams
    )

    train_input_fn = data_input_fn(train_filenames[0], num_input, batch_size=batch_size)
    eval_input_fn = data_input_fn(validation_filenames[0], num_input, batch_size=batch_size)
    
    experiment = tf.contrib.learn.Experiment(
        mnist_estimator,
        train_input_fn=train_input_fn,
        eval_input_fn=eval_input_fn,
        train_steps=num_steps,
        min_eval_frequency=10,
        eval_hooks=[summary_hook]
    )

    experiment.train_and_evaluate()
    
    experiment.evaluate()

## Hyperparameter search using `grid_params` method <a class="anchor" id='hyperparam'></a>

Hyperparameter optimization is critical to achieve the best accuracy for your model. With Hops, hyperparameter optimization is easier than ever.  We can find the best hyperparameters to train the model and make it easy to find the best set of hyper parameters by visualizing them in TensorBoard for you.

To define the hyperparameters you wish to perform gridsearch with, simply create a dictionary with the keys matching the arguments of your wrapper function, and a list of values for each parameter like below.

```python
# Arguments to use in training
args_dict = {'learning_rate': [0.001, 0.0005, 0.0001], 'dropout': [0.45, 0.7]}
```

Next step is to generate all possible combinations, the grid, of the hyperparameters.
```python
# Create a grid with the specified arguments
args_dict_grid = util.grid_params(args_dict)
```

So in effect, this is interpreted as that you want to run your TensorFlow code with 6 different hyperparameter combinations as shown in the table.


| Job number | Learning rate | Dropout |
|:----------:|:-------------:|:-------:|
|      1     |     0.001     |   0.45  |
|      2     |     0.001     |   0.7   |
|      3     |     0.0005    |   0.45  |
|      4     |     0.0005    |   0.7   |
|      5     |     0.0001    |   0.45  |
|      6     |     0.0001    |   0.7   |

In [ ]:
from hops import util

#Define dict for hyperparameters
args_dict = {'learning_rate': [0.001, 0.0005, 0.0001], 'dropout': [0.45, 0.7]}

# Generate a grid for the given hyperparameters
args_dict_grid = util.grid_params(args_dict)

print(args_dict_grid)

## Running the training using `tflauncher` module <a class="anchor" id='starting'></a>

The last, and arguably most important module that we demonstrate from the `hops` module is called `tflauncher`. The `tflauncher` module provides a single function called `launch`. As arguments it simply takes the `spark` SparkSession object, which is automatically created when the first cell is evaluated in the notebook, in addition to the wrapper function and the dictionary with the hyperparameters. `tflauncher.launch` will simply run the wrapper function and inject the value of each hyperparameter that you have specified. Assuming that you started Jupyter with the default configuration, each job will be run sequentially. To increase throughput by running two or more jobs in parallel, please see [this](#parallel).

In [ ]:
from hops import tflauncher

tensorboard_hdfs_logdir = tflauncher.launch(spark, wrapper, args_dict_grid)

## Monitoring execution - TensorBoard <a class="anchor" id='tensorboard'></a>
To find the TensorBoard for the execution, please go back to HopsWorks and follow the arrows in the images below.

![Image7-Monitor.png](../images/jupyter.png)

![Image4-LaunchTensorboard.png](../images/overview.png)

![Image8-Tensorboard.png](../images/tensorboard.png)

## Monitoring execution - logs <a class="anchor" id='logs'></a>
To find the logs for the execution, please go back to HopsWorks and follow the arrows in the images below.

![Image6-GoToLogs.png](../images/logs.png)

![Image9-Logs.png](../images/viewlogs.png)

## Increasing throughput - Running jobs in parallel <a class="anchor" id='parallel'></a>
The default configuration is to run each job sequentially, so each one of the 6 hyperparameter configuration is run sequentially. It is possible to increase the number of jobs that can be executed at any time by increasing the value of the configuration property shown in the picture below. To do this you need to restart Jupyter, by navigating back to HopsWorks and shutting it down.
![parallel.png](../images/parallel.png)

## Visualizing TensorBoard events from previous runs <a class="anchor" id='visualize'></a>

In [ ]:
from hops import tensorboard

# Visualize all TensorBoard events for the jobs in the same TensorBoard
tensorboard.visualize(spark, tensorboard_hdfs_logdir)

![image12-TensorboardAgg.png](../images/visualize.png)

## Moving on from here: Scaling out with TensorFlowOnSpark and Horovod

We walked through how to run one of most popular deep learning architectures, Cifar-10 dataset on hops using hops `hdfs` and `tensorboard` utility modules. 

Next step is to reduce training time by training the model in parallel. We covered in-depth tutorials about different parallelization concepts in the other notebooks. 